<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [1]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Caio Montes Correia, 21.00931-7' #@param {type:"string"}






In [4]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'
import pandas as pd

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [5]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [6]:
host_counts = df.groupby('host_name')['id'].nunique()
top_hosts = host_counts.nlargest(3)
top_hosts

host_name
Mark            7
Sonder (NYC)    7
Andrew          6
Name: id, dtype: int64

# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [16]:
neighbourhood_counts = df.groupby('neighbourhood')['id'].nunique()
neighbourhood_20_imoveis = neighbourhood_counts[neighbourhood_counts > 20]

df_20 = df[df['neighbourhood'].isin(neighbourhood_20_imoveis.index)]

preco_medio = df_20['price'].mean()

In [17]:
...

print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:.2f}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD 151.42


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [124]:
df_brooklyn = df[df['neighbourhood_group'] == 'Brooklyn']
df_brooklyn_outliers = df_brooklyn.copy()

Q1 = df_brooklyn_outliers['price'].quantile(0.25)
Q3 = df_brooklyn_outliers['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_brooklyn_outliers['outlier'] = df_brooklyn_outliers.apply(lambda row: 'True' if row['price'] < lower_bound or row['price'] > upper_bound else 'False', axis=1)


normal = df_brooklyn_outliers[df_brooklyn_outliers['outlier'] == 'False']
outlier = df_brooklyn_outliers[df_brooklyn_outliers['outlier'] == 'True']

percentualOutliers = len(outlier)* 100 / (len(normal) + len(outlier))

percentualOutliers

perc_Brooklyn= percentualOutliers

In [125]:
...

print( f'Percentuais de outliers de preços, Brooklyn: {perc_Brooklyn:.2f} %')


Percentuais de outliers de preços, Brooklyn: 5.76 %


# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [63]:
df_scores = df.copy()

df_scores['availability_score'] = df_scores['availability_365'].apply(lambda row: 10 if row > 90 else -5 if row < 5 else 0)
df_scores['review_score'] = df_scores['number_of_reviews'].apply(lambda row: 5 * (row // 10) if row >= 10 else -5 if row == 0 else 0)
df_scores['neighbourhood_mean_price'] = df_scores.groupby('neighbourhood')['price'].transform('mean')
df_scores['price_score'] = df_scores.apply(lambda row: 5 if row['price'] < row['neighbourhood_mean_price'] else 0, axis=1)


df_scores['score'] = df_scores['availability_score'] + df_scores['review_score'] + df_scores['neighbourhood_mean_price'] + df_scores['price_score']

In [64]:
...

print(f'A soma dos valores de score é: ', df_scores['score'].sum())


A soma dos valores de score é:  134752.0


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [75]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes
df_airbnb = df.copy()

df_taxes_pivot = df_taxes.pivot_table(index = 'id', values='value', columns='type', observed=True).reset_index().fillna(0)

df_enriquecido = df_airbnb.merge(df_taxes_pivot, on = 'id', how = 'left')

mean_taxes = df_enriquecido.groupby('neighbourhood_group')['taxes'].mean()
mean_fines = df_enriquecido.groupby('neighbourhood_group')['fines'].mean()

print(int(mean_taxes.loc['Manhattan']))
print(int(mean_fines.loc['Manhattan']))

1008
357


# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [147]:
df_airbnb = df.copy()
df_report = df_airbnb.sort_values(by=['neighbourhood_group', 'neighbourhood']).reset_index()



df_report['line_id'] = df_report.groupby(['neighbourhood_group']).cumcount()


In [129]:
df_report[['neighbourhood_group','neighbourhood','price','page_id','line_id']].head(60)

neighbourhood_group       neighbourhood  price  page_id  line_id
0                Bronx             Belmont     29        0        0
1                Bronx   Claremont Village     67        0        1
2                Bronx           Concourse     40        0        2
3                Bronx            Edenwald     90        0        3
4                Bronx            Edenwald     80        1        4
5                Bronx           Fieldston     85        0        5
6                Bronx           Fieldston    109        1        6
7                Bronx             Fordham     79        0        7
8                Bronx          Highbridge     22        0        8
9                Bronx             Melrose     69        0        9
10               Bronx         Morris Park     40        0       10
11               Bronx          Mott Haven    100        0       11
12               Bronx         Port Morris     28        0       12
13               Bronx           Riverdale     99        0       13
14               Bronx            Van Nest    107        0       14
15               Bronx           Wakefield    120        0       15
16               Bronx           Wakefield    120        1       16
17               Bronx      Williamsbridge     45        0       17
18               Bronx      Williamsbridge     65        1       18
19               Bronx      Williamsbridge     36        2       19
20            Brooklyn           Bay Ridge    100        0        0
21            Brooklyn           Bay Ridge   1800        1        1
22            Brooklyn  Bedford-Stuyvesant    125        0        2
23            Brooklyn  Bedford-Stuyvesant    250        1        3
24            Brooklyn  Bedford-Stuyvesant     59        2        4
25            Brooklyn  Bedford-Stuyvesant     45        3        5
26            Brooklyn  Bedford-Stuyvesant     55        4        6
27            Brooklyn  Bedford-Stuyvesant     57        5        7
28            Brooklyn  Bedford-Stuyvesant     59        6        8
29            Brooklyn  Bedford-Stuyvesant    150        7        9
30            Brooklyn  Bedford-Stuyvesant     60        8       10
31            Brooklyn  Bedford-Stuyvesant    100        9       11
32            Brooklyn  Bedford-Stuyvesant     50       10       12
33            Brooklyn  Bedford-Stuyvesant    150       11       13
34            Brooklyn  Bedford-Stuyvesant     70       12       14
35            Brooklyn  Bedford-Stuyvesant     65       13       15
36            Brooklyn  Bedford-Stuyvesant     45       14       16
37            Brooklyn  Bedford-Stuyvesant     70       15       17
38            Brooklyn  Bedford-Stuyvesant     90       16       18
39            Brooklyn  Bedford-Stuyvesant    225       17       19
40            Brooklyn  Bedford-Stuyvesant    150       18       20
41            Brooklyn  Bedford-Stuyvesant    155       19       21
42            Brooklyn  Bedford-Stuyvesant     30       20       22
43            Brooklyn  Bedford-Stuyvesant    250       21       23
44            Brooklyn  Bedford-Stuyvesant     33       22       24
45            Brooklyn  Bedford-Stuyvesant     50       23       25
46            Brooklyn  Bedford-Stuyvesant     50       24       26
47            Brooklyn  Bedford-Stuyvesant     55       25       27
48            Brooklyn  Bedford-Stuyvesant    150       26       28
49            Brooklyn  Bedford-Stuyvesant     55       27       29
50            Brooklyn  Bedford-Stuyvesant     50       28       30
51            Brooklyn  Bedford-Stuyvesant     69       29       31
52            Brooklyn  Bedford-Stuyvesant     55       30       32
53            Brooklyn  Bedford-Stuyvesant     55       31       33
54            Brooklyn  Bedford-Stuyvesant     55       32       34
55            Brooklyn  Bedford-Stuyvesant     99       33       35
56            Brooklyn  Bedford-Stuyvesant    180       34       36
57            Brooklyn  Bedford-Stuyvesant      0       35 

In [ ]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,26,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7